In [ ]:
import os
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import package.myUtil as myUtil

In [ ]:
n = 128
m = 128
DATA_PATH = "../../OneDrive - m.titech.ac.jp/Lab/data"
# DATA_PATH = "../data"
DIRECTORY = DATA_PATH + "/241112"
SETTING = f"gf"

if not os.path.exists(DIRECTORY):
    os.makedirs(DIRECTORY)
if not os.path.exists(DIRECTORY + "/systemMatrix"):
    os.makedirs(DIRECTORY + "/systemMatrix")

In [ ]:
# load images
IMG_NAME = "hadamard"
# INFO = "cap_R_230516_128"
CAP_DATE = "241107"
use_list = myUtil.get_use_list(n*n, 1.0)
G = myUtil.images2matrix(f"{DATA_PATH}/{IMG_NAME}{n}_cap_{CAP_DATE}/", use_list, thin_out=True)
F = myUtil.images2matrix(f"{DATA_PATH}/{IMG_NAME}{n}_input/", use_list).astype(cp.int8)
M, K = G.shape
N, K = F.shape
print("G shape:", G.shape, "F shape:", F.shape, "M=", M, "N=", N, "K=", K)
print("G max:", G.max(), "G min:", G.min(), "F max:", F.max(), "F min:", F.min())

black_img = Image.open(f"{DATA_PATH}/capture_{CAP_DATE}/Black.png").convert("L")
black = cp.asarray(black_img) / 255
black = black[::2, ::2].ravel()
B = cp.tile(black[:, None], K)

G = G - B

white_img = Image.open(f"{DATA_PATH}/capture_{CAP_DATE}/White.png").convert("L")
white = cp.asarray(white_img) / 255
white = white[::2, ::2].ravel() - black
H1 = cp.tile(white[:, None], K)

F_hat = (2 * F - 1).astype(cp.int8)
G_hat = 2 * G - H1
del F, G, H1

In [ ]:
H = G_hat @ F_hat.T / N

In [ ]:
# np.save(f"{DIRECTORY}/systemMatrix/H_matrix_{SETTING}.npy", H.get())
# print(f"Saved {DIRECTORY}/systemMatrix/H_matrix_{SETTING}.npy")

In [ ]:
SAMPLE_NAME = "Cameraman"
sample_image = Image.open(f"{DATA_PATH}/sample_image{n}/{SAMPLE_NAME}.png").convert('L')
sample_image = cp.asarray(sample_image).ravel() / 255

FILENAME = f"{SAMPLE_NAME}_{SETTING}_b.png"
Hf = H @ sample_image + black
Hf_img = cp.asnumpy(Hf.reshape(m, m))
Hf_img = np.clip(Hf_img, 0, 1)
Hf_pil = Image.fromarray((Hf_img * 255).astype(np.uint8), mode='L')
Hf_pil.save(f"{DIRECTORY}/{FILENAME}", format='PNG')
print(f"Saved {DIRECTORY}/{FILENAME}")
Hf_pil

In [ ]:
plt.imshow(Hf_img, cmap='gray', interpolation='nearest')
plt.colorbar()
plt.title('Grayscale Heatmap')
plt.show()